In [1]:
import time

import keras_tuner
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Conv2D, MaxPool2D
from tensorflow.keras.models import Sequential

from methods_audio import data_handling

/Users/rosameliacarioni/miniconda3/envs/bach_thesis_4/lib/python3.9/site-packages/tensorflow_io-0.32.0-py3.9-macosx-11.0-arm64.egg/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/Users/rosameliacarioni/miniconda3/envs/bach_thesis_4/lib/python3.9/site-packages/tensorflow_io-0.32.0-py3.9-macosx-11.0-arm64.egg/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ["[Errno 2] The file to load file system plugin from does not exist.: '/Users/rosameliacarioni/miniconda3/envs/bach_thesis_4/lib/python3.9/site-packages/tensorflow_io-0.32.0-py3.9-macosx-11.0-arm64.egg/tensorflow_io/python/ops/libtensorflow_io_plugins.so'"]
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/Users/rosameliacarioni/miniconda3/envs/bach_thesis_4/lib/python3.9/site-packages/tensorflow_io-0.32.0-py3.9-macosx-11.0-arm64.egg/tensorflow_io/python/ops/__init__.py:104: UserWarning

# 1. Get data (file names)

In [2]:
data = data_handling.get_data()

Metal device set to: Apple M1 Pro

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



# 2. Read data (transforming file names into waves) <br>
Additionally, the mean is removed and the data is normalized. 

In [3]:
data = data.map(data_handling.read_in_data)

# 3. Get input for model training 

In [4]:
samples, labels = data_handling.extract_samples_labels(data)

2023-05-14 16:26:43.923211: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


# 4. Split data into train and validation sets

In [5]:
validation_set_size = 0.20
x_train, x_valid, y_train, y_valid = train_test_split(samples, labels, test_size=validation_set_size, random_state=123)

# 5. Transform data to spectograms

In [6]:
type_transformation = "mfcc"
x_train = data_handling.transform_data(x_train, type_transformation)
x_valid = data_handling.transform_data(x_valid, type_transformation)

/Users/rosameliacarioni/University/Thesis/code/methods_audio/data_handling.py:192: FutureWarning: Pass y=[ 0.          0.          0.         ... -0.00044566  0.00122628
 -0.00253558] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mfccs = librosa.feature.mfcc(wave.numpy(), n_mfcc = 13, sr = sr)
/Users/rosameliacarioni/University/Thesis/code/methods_audio/data_handling.py:192: FutureWarning: Pass y=[ 0.          0.          0.         ... -0.02806111 -0.02002473
 -0.02806111] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mfccs = librosa.feature.mfcc(wave.numpy(), n_mfcc = 13, sr = sr)
/Users/rosameliacarioni/University/Thesis/code/methods_audio/data_handling.py:192: FutureWarning: Pass y=[0.         0.         0.         ... 0.18384257 0.1622837  0.25262564] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mfccs = librosa.feature.mfcc

## 6. Build model with hyperparameter tunning 
https://keras.io/guides/keras_tuner/getting_started/ <br>
https://www.youtube.com/watch?v=6Nf1x7qThR8&ab_channel=GregHogg 

In [13]:
def build_model(hp):
    input = (13, 157, 1)
    model = keras.Sequential()

    # Add input layer
    # matching samples.shape
    model.add(
        Conv2D(
            filters=hp.Int("conv_filters_0", min_value=8, max_value=128, step=16),
            activation=hp.Choice("conv_activation_0", ["relu", "tanh"]),
            kernel_size=(3, 3),
            padding="same",
            input_shape=input,
        )
    )
    model.add(MaxPool2D(pool_size=(2, 2)))

    # Tune the number of Conv layers
    # padding = "same" means that output has same size as the input, and not max pooling is applied in this case because the input is already small in size
    for i in range(hp.Int("num_conv_layers", 1, 2)):
        model.add(
            Sequential(
                [
                    layers.Conv2D(
                        filters=hp.Int(f"conv_filters_{i}", min_value=8, max_value=128, step=16),
                        activation=hp.Choice(f"conv_activation_{i}", ["relu", "tanh"]),
                        kernel_size=(4, 3),
                        padding="same",
                    ),
                    layers.MaxPool2D(pool_size=(2, 2)),
                ]
            )
        )
    model.add(layers.Flatten())

    # Tune the number of Dense layers and Tune whether to use dropout layer
    for i in range(hp.Int("num_dense_layers", 1, 6)):
        model.add(
            Sequential(
                [
                    layers.Dense(
                        # Tune number of units separately.
                        units=hp.Int(f"dense_units_{i}", min_value=50, max_value=600, step=50),
                        activation=hp.Choice(f"dense_activation_{i}", ["relu", "tanh"]),
                    ),
                    layers.Dropout(rate=hp.Float(f"dense_dropout_{i}", min_value=0, max_value=1)),
                ]
            )
        )

    model.add(
        layers.Dense(
            units=1,  # because we have 2 classes
            activation=hp.Choice("activatio_last_layer", ["softmax", "sigmoid"]),
        )
    )

    # Define the optimizer learning rate as a hyperparameter.
    # sampling="log", the step is multiplied between samples.
    lr = hp.Float("learning_rate", min_value=1e-4, max_value=1e-1, sampling="log")
    model.compile(
        optimizer=keras.optimizers.SGD(learning_rate=lr),
        loss="BinaryCrossentropy",
        metrics=["accuracy"],
    )

    return model

In [14]:
build_model(keras_tuner.HyperParameters())

### Initialize tuner by specifying different arguments 

In [15]:
tuner = keras_tuner.BayesianOptimization(
    hypermodel=build_model,
    objective="val_accuracy",  # we want maximize accuracy
    max_trials=50,
    overwrite=True,
    directory="param_optimization",
    project_name="mfcc",
)

In [16]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=5)
# patience refers to number of epochs: if the val loss is not improving fter 5 ephocs, we stop it.

### During the search, the model is called with different hyperparameters 

In [17]:
tuner.search_space_summary()
# Default search space size: number of hyper parameters that we are tunning

Search space summary
Default search space size: 9
conv_filters_0 (Int)
{'default': None, 'conditions': [], 'min_value': 8, 'max_value': 128, 'step': 16, 'sampling': 'linear'}
conv_activation_0 (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'tanh'], 'ordered': False}
num_conv_layers (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 2, 'step': 1, 'sampling': 'linear'}
num_dense_layers (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 6, 'step': 1, 'sampling': 'linear'}
dense_units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 50, 'max_value': 600, 'step': 50, 'sampling': 'linear'}
dense_activation_0 (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'tanh'], 'ordered': False}
dense_dropout_0 (Float)
{'default': 0.0, 'conditions': [], 'min_value': 0.0, 'max_value': 1.0, 'step': None, 'sampling': 'linear'}
activatio_last_layer (Choice)
{'default': 'softmax', 'conditions': [], 'values': ['softmax', 'sigmo

In [18]:
epochs = 50
start_time = time.time()

tuner.search(
    np.stack(x_train),
    np.stack(y_train),
    epochs=epochs,
    validation_data=(np.stack(x_valid), np.stack(y_valid)),
    callbacks=[stop_early],
)  # similar to fit

end_time = time.time()

Trial 50 Complete [00h 00m 39s]
val_accuracy: 0.49648380279541016

Best val_accuracy So Far: 0.9606188535690308
Total elapsed time: 00h 44m 10s
INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit


In [19]:
elapsed_time = end_time - start_time
print(f"The search took {elapsed_time:.2f} seconds to finish.")

The search took 2649.91 seconds to finish.


In [20]:
tuner.results_summary()

Results summary
Results in param_optimization/mfcc
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 39 summary
Hyperparameters:
conv_filters_0: 72
conv_activation_0: tanh
num_conv_layers: 2
num_dense_layers: 6
dense_units_0: 500
dense_activation_0: relu
dense_dropout_0: 0.022246604558724847
activatio_last_layer: sigmoid
learning_rate: 0.019211177831550998
dense_units_1: 50
dense_activation_1: relu
dense_dropout_1: 0.20665118185955877
dense_units_2: 500
dense_activation_2: relu
dense_dropout_2: 0.24414632407615067
dense_units_3: 450
dense_activation_3: relu
dense_dropout_3: 0.46621231837096866
dense_units_4: 350
dense_activation_4: tanh
dense_dropout_4: 0.8544439888617841
dense_units_5: 200
dense_activation_5: tanh
dense_dropout_5: 0.7849894331383263
conv_filters_1: 72
conv_activation_1: relu
Score: 0.9606188535690308

Trial 35 summary
Hyperparameters:
conv_filters_0: 104
conv_activation_0: relu
num_conv_layers: 2
num_dense_layers: 5
dense_units_0: 200
dense

### After all of that we don't have a model yet but rather a set of hyper parameters. Let's query the results and create a model:

In [21]:
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print(best_hps.values)

{'conv_filters_0': 72, 'conv_activation_0': 'tanh', 'num_conv_layers': 2, 'num_dense_layers': 6, 'dense_units_0': 500, 'dense_activation_0': 'relu', 'dense_dropout_0': 0.022246604558724847, 'activatio_last_layer': 'sigmoid', 'learning_rate': 0.019211177831550998, 'dense_units_1': 50, 'dense_activation_1': 'relu', 'dense_dropout_1': 0.20665118185955877, 'dense_units_2': 500, 'dense_activation_2': 'relu', 'dense_dropout_2': 0.24414632407615067, 'dense_units_3': 450, 'dense_activation_3': 'relu', 'dense_dropout_3': 0.46621231837096866, 'dense_units_4': 350, 'dense_activation_4': 'tanh', 'dense_dropout_4': 0.8544439888617841, 'dense_units_5': 200, 'dense_activation_5': 'tanh', 'dense_dropout_5': 0.7849894331383263, 'conv_filters_1': 72, 'conv_activation_1': 'relu'}


In [22]:
learning_rate = best_hps.get("learning_rate")
print(learning_rate)

0.019211177831550998


In [23]:
model = tuner.hypermodel.build(best_hps)

In [24]:
location = "data/models/mfcc"
model.save(location)

INFO:tensorflow:Assets written to: data/models/mfcc/assets


INFO:tensorflow:Assets written to: data/models/mfcc/assets
